In [ ]:
#필요한 모듈 설치하기
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 46.9 MB 43 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.4 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595734 sha256=e0c518a9d52497fda0f008695db4c9172658245a9eb578ca462e73e456da7c47
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.3 MB/s 
     |████████████████████████████████| 769 kB 5.5 MB/s 
     |████████████████████████████████| 895 kB 33.0 MB/s 
     |████████████████████████████████| 3.0 MB 19.1 MB/s 


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-s2m92vvb
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-s2m92vvb
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=52a060b18d8b88bc107cbfe22855735b5425d316799fedce62187b8167fcc09a
  Stored in directory: /tmp/pip-ephem-wheel-cache-xup0wvvr/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [ ]:
#kobert라이브러리에서 많이 쓰이는 함수 불러오기
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
#transformers에서 하이퍼파라미터 세팅
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용(권장)
device = torch.device("cuda:0")
#CPU 사용
#device = torch.device("cpu")

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
#구글 드라이브에 저장되어있던 데이터 불러오기
chatbot_data_pre = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/감성대화.xlsx')
#chatbot_data_new = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/한국어_단발성_대화_데이터셋.xlsx')

In [ ]:
len(chatbot_data_pre)

9974

In [ ]:
#데이터전처리
#데이터구조화 -> 정신건강기반 의미구축 모델을 기반으로 세부감정 -> 분노,불안,우울,자살,긍정 카테고리화

#0  기쁨
#1. 분노 - 안달하는 성가신 툴툴대는 방어적인 혐오스러운
#2. 우울 - 비통한 슬픔 우울한 실망한 눈물이나는 후회되는  버려진
#3. 불안 - 걱정스러운 조심스러운 초조한 두려운 스트레스받는
#4. 자살 - 고립된 상처 염세적인

chatbot_data = chatbot_data_pre.loc[(chatbot_data_pre['감정_대분류'] == "기쁨") | (chatbot_data_pre['감정_소분류'] == "안달하는") | (chatbot_data_pre['감정_소분류'] == "성가신") | (chatbot_data_pre['감정_소분류'] == "툴툴대는") | (chatbot_data_pre['감정_소분류'] == "방어적인") | (chatbot_data_pre['감정_소분류'] == "혐오스러운")
 | (chatbot_data_pre['감정_소분류'] == "비통한") | (chatbot_data_pre['감정_소분류'] == "슬픔") | (chatbot_data_pre['감정_소분류'] == "우울한") | (chatbot_data_pre['감정_소분류'] == "실망한") | (chatbot_data_pre['감정_소분류'] == "후회되는") | (chatbot_data_pre['감정_소분류'] == "눈물이 나는")
 | (chatbot_data_pre['감정_소분류'] == "걱정스러운") | (chatbot_data_pre['감정_소분류'] == "조심스러운") | (chatbot_data_pre['감정_소분류'] == "초조한") | (chatbot_data_pre['감정_소분류'] == "두려운") | (chatbot_data_pre['감정_소분류'] == "스트레스 받는")
 | (chatbot_data_pre['감정_소분류'] == "고립된") | (chatbot_data_pre['감정_소분류'] == "염세적인") | (chatbot_data_pre['감정_소분류'] == "상처") 
 ]

len(chatbot_data)

4852

In [ ]:
#데이터 구조 살펴보기
chatbot_data.sample(n=10)

,연령,성별,상황키워드,감정_대분류,감정_소분류,사람문장1,기계문장1,사람문장2,기계문장2,사람문장3,기계문장3
1311,청소년,남성,가족관계,슬픔,비통한,오늘 동생이 과자를 사달라고 했는데 내가 돈이 없어서 못 사줬어. 너무 미안해.,동생이 먹고 싶은 과자를 못 사주셔서 마음이 편치 않았겠어요.,동생은 돈이 없다는 개념도 아직 어려서 잘 모를 거야. 그게 더 마음이 아프다.,예전에도 이런 적이 있었나요? 어떻게 하셨나요?,집에 있는 다른 과자를 줬어. 오늘도 그래야겠다.,집에 있는 과자로 문제를 해결하실 생각이시군요.
3156,청년,남성,"연애,결혼,출산",당황,고립된,아내와 이혼하고 나니 아이들까지 나를 무시하기 시작해서 너무 당황스러워.,어떤 것이 당황스럽게 만들었나요?,아이들이 전처럼 나를 따르지도 않고 나를 싫어하는 것 같아.,자녀분들이 전처럼 따르지 않아서 당황스러웠군요. 어떻게 하면 상황을 변화시킬 수 있...,아이들과 진지하게 이야기를 해봐야겠어.,자녀분들과의 대화로 사이가 좋아지셨으면 좋겠어요.
7925,노년,여성,"건강, 죽음",슬픔,눈물이 나는,오늘 우연히 창밖을 보았는데 학생들이 친구들과 함께 즐겁게 노는 모습을 보았더니 슬펐어.,무엇이 슬픔을 느끼도록 만들었나요?,친구들과 함께 있는 모습을 보니 세상을 떠난 내 친구들이 생각나더라.,친구들이 떠나 슬프시겠어요.,오늘은 아직 남아있는 친구들을 보러 가야겠어.,친구들과 즐거운 시간 되었으면 좋겠어요.
4609,청년,여성,"연애,결혼,출산",분노,툴툴대는,남자 친구는 나와 쇼핑하는 게 즐겁지 않은가 봐. 계속 핸드폰만 보고 있어서 서운하...,남자 친구가 함께 쇼핑하는 것에 흥미가 없어서 서운하시군요.,난 같이 얘기하면서 즐겁게 쇼핑하고 싶었는데 남자 친구는 그렇지 않은가 봐.,어떻게 해야 함께 즐겁게 쇼핑을 할 수 있을까요?,남자 친구가 관심 있어 하는 곳도 둘러보며 쇼핑해봐야겠어.,서로 즐거운 쇼핑해 되셨으면 좋겠네요.
1348,청소년,남성,학교폭력/따돌림,슬픔,우울한,내가 왕따를 당하는데 내 상황을 들어주는 애가 없네.,옆에 공감해줄 사람이 없으니 힘드시겠어요.,그러니까. 앞으로 어떻게 해야 할지 암담하네.,그러게요. 어떻게 하면 이 상황에서 해결될까요?,아무래도 청소년 상담 사이트에 가서 온라인 상담을 받아야겠어.,상담을 통해서 마음이 편해졌으면 좋겠어요.
2837,청년,여성,"진로,취업,직장",기쁨,흥분,몇 년 동안 준비했던 공무원 필기시험에 합격했어. 너무 기뻐,시험에 합격하셔서 매우 기쁘시겠어요.,부모님에게 이 사실을 당장 말해주고 싶은데 뭐라고 말해야 될까?,무슨 말을 하는 것이 가장 좋을까요?,사랑한다는 말을 하고 싶어. 내가 시험 보는 동안 열심히 응원해주셨거든,사랑한다는 말을 통해 가족 모두 기쁨을 나눌 수 있으면 좋겠네요.
8857,노년,남성,"건강, 죽음",기쁨,안도,돈이 없는 상황에서 친구가 아내의 병원비를 빌려주겠다고 했어.,친구가 돈을 빌려준다니 정말 기쁘겠어요.,응. 전전긍긍하던 차에 친구가 도와준다고 해서 안도했어.,이러한 상황에서 어떻게 해야 바람직할까요?,친구가 도움이 필요한 상황에 처하면 내가 나서서 도와야겠어.,원하는 대로 꼭 하시길 바랄게요.
5264,중년,남성,"대인관계(부부, 자녀)",기쁨,느긋,자식들은 다 커서 앞가림을 할 나이가 되었고 우리도 노후 준비가 되어있어 한결 여유...,자녀들이 앞가림을 하고 노후 준비가 되어 있어 여유롭고 기쁘시군요.,아이들이 어렸을 때는 우물가에 내놓은 것 마냥 항상 불안했는데 어느덧 아이들이 자랑...,아이들이 어렸을 때는 불안했는데 지금은 자랑스러우시군요. 지금의 상황을 어떻게 하면...,앞으로는 아이들을 좀 더 멀리서 지켜봐 주고 응원해주려고 해.,앞으로도 항상 아이들을 응원해주는 멋진 부모가 되시길 바라요.
2671,청년,남성,"연애,결혼,출산",기쁨,안도,여자 친구가 내게 거짓말하지 않았다는 사실을 알고 안도했어.,여자 친구가 거짓말하지 않았다는 사실에 안도하셨군요.,여자 친구가 전에 내게 친구와 있다고 거짓말을 한 적이 있었어.,비슷한 일이 있으셨군요. 지금의 상황에서 할 수 있는 게 어떤 것이 있을까요?,더 이상 고민하지 말고 여자 친구의 말을 신뢰하도록 해야겠어.,여자 친구에 대한 믿음으로 잘 해결했으면 좋겠어요.
9113,노년,남성,"건강, 죽음",기쁨,자신하는,몸을 움직이는게 불편해서 병원에 갔는데 내가 당뇨병에 걸렸데.,당뇨병은 몸을 꾸준히 신경써서 관리해야 하는데 힘드시겠네요.,그렇지 하지만 나는 오래살고 싶으니까 그 정도는 충분히 할 수 있어.,그렇다면 주로 무엇을 하면서 건강을 관리할 계획이신가요?,자주 바깥에 나가 걷고 탄수화물이나 당이 들어간 음식은 적게 먹는 것 부터 시작할꺼야.,노력하신 만큼 오랫동안 건강하게 지내셨으면 좋겠네요.


In [ ]:
# 감정데이터 정수형(수치)로 바꿔주기 (정수 인코딩)
chatbot_data.loc[(chatbot_data_pre['감정_소분류'] == "안달하는") | (chatbot_data_pre['감정_소분류'] == "성가신") | (chatbot_data_pre['감정_소분류'] == "툴툴대는") | (chatbot_data_pre['감정_소분류'] == "방어적인") | (chatbot_data_pre['감정_소분류'] == "혐오스러운"), 'Emotion'] = 1  #분노 => 1
chatbot_data.loc[(chatbot_data_pre['감정_소분류'] == "비통한") | (chatbot_data_pre['감정_소분류'] == "슬픔") | (chatbot_data_pre['감정_소분류'] == "우울한") | (chatbot_data_pre['감정_소분류'] == "실망한") | (chatbot_data_pre['감정_소분류'] == "후회되는") | (chatbot_data_pre['감정_소분류'] == "눈물이 나는"), 'Emotion'] = 2  #우울 => 2
chatbot_data.loc[(chatbot_data_pre['감정_소분류'] == "걱정스러운") | (chatbot_data_pre['감정_소분류'] == "조심스러운") | (chatbot_data_pre['감정_소분류'] == "초조한") | (chatbot_data_pre['감정_소분류'] == "두려운") | (chatbot_data_pre['감정_소분류'] == "스트레스 받는"), 'Emotion'] = 3  #불안 => 3
chatbot_data.loc[(chatbot_data_pre['감정_소분류'] == "고립된") | (chatbot_data_pre['감정_소분류'] == "염세적인") | (chatbot_data_pre['감정_소분류'] == "상처")  , 'Emotion'] = 4  #자살 => 4
chatbot_data.loc[(chatbot_data['감정_대분류'] == "기쁨"), 'Emotion'] = 0  #행복 => 5

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [ ]:
# 이중리스트형식으로 data_list에 감정과 텍스트 담기
data_list = []
for q, label in zip(chatbot_data['사람문장1'], chatbot_data['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(int(label)))

    data_list.append(data)

In [ ]:
# 자살 데이터 추가
extra_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/AI_WEB_AICARE_AIM/data_preprocesssing/mental_illness_caterogies/자살.csv", encoding='cp949', header=None)
extra_data[1] = extra_data[1].astype(str)
data_list += [list(i) for i in extra_data.iloc]
len(data_list)

5002

In [ ]:
# data_list에 감정,텍스트 형식 살펴보기
print(data_list[0])
print(data_list[600])
print(data_list[1200])
print(data_list[1800])
print(data_list[2400])
print(data_list[4900])
print(data_list[-1])

['오늘 반장 선거에서 내가 반장이 됐어! 친구들이 날 믿어줘서 너무 고맙다.', '0']
['선생님과 진로 상담을 했는데 형식적인 대화만 하고 속마음은 잘 얘기하지 않았어.', '1']
['가족과 같이 있는데 나 혼자 있는 기분이라서 속상해.', '4']
['내가 시댁하고 속상한 일이 생기면 남편은 시댁의 편만 들어서 서운해.', '4']
['프로젝트를 반드시 기한 내에 끝내기 위해 팀원들을 닥달하고 있어.', '1']
['운전하다가 확 낭떠러지로 떨어졌으면 좋겠어', '4']
['하 죽고 싶어 힘들어', '4']


In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

# 훈련,테스트데이터 개수 확인
print(len(dataset_train))
print(len(dataset_test))

3751
1251


In [ ]:
# BERT 모델에 들어가기 위한 dataset을 만들어주는 클래스
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# 파라미터 세팅 ( 문장길이, 모델배치수,에폭수 ...)
max_len = 50
batch_size = 64
warmup_ratio = 0.1
num_epochs = 12
max_grad_norm = 1
log_interval = 200
learning_rate =  3e-5

In [ ]:
#문장을 토크나이저를 통해서 토큰으로 변환(토큰화)
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
# 데이터셋 BERT에 맞게 적용
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
# 텍스트의 토큰화 -> 정수인코딩 결과
data_train[0]

(array([   2, 1425, 7096,  806, 3084, 6037, 7245, 6553, 1458, 5655, 2514,
        7636, 7828, 3060, 7227, 7096, 6844,  517,   54,    3,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1], dtype=int32),
 array(20, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0], dtype=int32),
 2)

In [ ]:
#bert모델에 넣기위해 torch형식으로 변환
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# KoBERT 학습모델 만들기
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.6).to(device)

In [ ]:
#optimizer파라미터 세팅
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
# 옵티마이저설정(AdamW) 추후에 다른 옵티마이저 확인 필요
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
#스케줄 설정
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
# 훈련모델의 정확도 계산함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
# 훈련데이터 형식 확인 
train_dataloader

In [ ]:
#데이터 학습모델을 통해 학습 and 테스트
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/59 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.6586742401123047 train acc 0.21875
epoch 1 train acc 0.5554446436332029


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 11 test acc 0.8327232142857144


  0%|          | 0/59 [00:00<?, ?it/s]

epoch 12 batch id 1 loss 0.015947287902235985 train acc 1.0
epoch 12 train acc 0.9936440677966102


  0%|          | 0/20 [00:00<?, ?it/s]

epoch 12 test acc 0.8335044642857143


In [ ]:
#classification_report 만들기위해 데이터 전처리
x_pred = []
x_label = []
y_test = []
for i in dataset_test:
  x_pred.append(int(i[1]))
  x_label.append(str(i[0]))

test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)
model.eval()

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)

    valid_length= valid_length
    label = label.long().to(device)

    out = model(token_ids, valid_length, segment_ids)

    test_eval=[]
    for i in out:
        logits=i
        logits = logits.detach().cpu().numpy()
        
        y_test.append(np.argmax(logits))

 # 데이터셋 각 라벨에 대한 정확도 측정
#y_predicted = bilstm_model.predict([X_test])
#pred_tags = f1score.sequences_to_tags(y_predicted)
#test_tags = f1score.sequences_to_tags(y_test)

#print(classification_report(test_tags, pred_tags))

# confusion_matrix 와 classification_report를 통해 결과값 정리
from sklearn.metrics import confusion_matrix, classification_report#

df = pd.DataFrame(columns=['Labels Data','Predicted Labels', 'Actual Labels'])
df['Labels Data'] = x_label
df['Predicted Labels'] = y_test
df['Actual Labels'] = x_pred

df[(df['Actual Labels']==4) & (df['Actual Labels']!=df['Predicted Labels'])].groupby('Predicted Labels').sum()

print(classification_report(x_pred, y_test,target_names=['기쁨','분노','우울','불안','자살']))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


              precision    recall  f1-score   support

          기쁨       0.98      0.95      0.96       421
          분노       0.81      0.72      0.76       210
          우울       0.79      0.78      0.78       258
          불안       0.79      0.85      0.82       191
          자살       0.69      0.79      0.73       171

    accuracy                           0.84      1251
   macro avg       0.81      0.82      0.81      1251
weighted avg       0.84      0.84      0.84      1251



In [ ]:
# 개별 문장을 넣어볼수 있는 (예측하는) 함수
y_test = []
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            #print(logits)
            if np.argmax(logits) == 0:
                test_eval.append("기쁨이")
            elif np.argmax(logits) == 1:
                test_eval.append("분노가")
            elif np.argmax(logits) == 2:
                test_eval.append("우울이")
            elif np.argmax(logits) == 3:
                test_eval.append("불안이")
            elif np.argmax(logits) == 4:
                test_eval.append("자살우려가")
            
        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

# 문장입력기
while True:
  sentence = input("하고싶은 말을 입력해주세요 : ")
  predict(sentence)
  print("\n")

하고싶은 말을 입력해주세요 : 오늘 진급해서 너무 기뻐


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 입력하신 내용에서 기쁨이 느껴집니다.


하고싶은 말을 입력해주세요 : 선임들때문에 너무 힘들어서 죽고싶어
>> 입력하신 내용에서 자살우려가 느껴집니다.


하고싶은 말을 입력해주세요 : 매일 너무 우울한 기분이 들어
>> 입력하신 내용에서 자살우려가 느껴집니다.


하고싶은 말을 입력해주세요 : 군생활이 아직도 너무 많이 남아서 슬퍼
>> 입력하신 내용에서 우울이 느껴집니다.


하고싶은 말을 입력해주세요 : 


EOFError: ignored

In [ ]:
ㅇ